In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter

# 定义超参数
batch_size = 64
learning_rate = 0.001
num_epochs = 5

# 数据预处理
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # MNIST 数据集的均值和标准差
])

# 加载 MNIST 数据集
train_dataset = datasets.MNIST(
    root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(
    root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset,
                         batch_size=batch_size, shuffle=False)

# 定义简单的 CNN 模型


class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)  # 展平
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# 初始化模型、损失函数和优化器
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 初始化 TensorBoard
writer = SummaryWriter('logs/mnist_experiment')

# 训练模型
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # 前向传播
        outputs = model(images)
        loss = criterion(outputs, labels)

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 记录损失到 TensorBoard
        writer.add_scalar('Training Loss', loss.item(),
                          epoch * len(train_loader) + i)

        if (i + 1) % 100 == 0:
            print(
                f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{total_step}], Loss: {loss.item():.4f}')

    # 每个 epoch 结束后在测试集上评估模型
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Test Accuracy after Epoch {epoch + 1}: {accuracy:.2f}%')
    writer.add_scalar('Test Accuracy', accuracy, epoch)

    model.train()

# 记录模型结构
images, _ = next(iter(train_loader))
writer.add_graph(model, images)


print("训练完成！")

Epoch [1/5], Step [100/938], Loss: 0.2888
Epoch [1/5], Step [200/938], Loss: 0.1367
Epoch [1/5], Step [300/938], Loss: 0.0563
Epoch [1/5], Step [400/938], Loss: 0.1753
Epoch [1/5], Step [500/938], Loss: 0.0923
Epoch [1/5], Step [600/938], Loss: 0.0816
Epoch [1/5], Step [700/938], Loss: 0.1019
Epoch [1/5], Step [800/938], Loss: 0.0091
Epoch [1/5], Step [900/938], Loss: 0.0359
Test Accuracy after Epoch 1: 98.56%
Epoch [2/5], Step [100/938], Loss: 0.0112
Epoch [2/5], Step [200/938], Loss: 0.0062
Epoch [2/5], Step [300/938], Loss: 0.0481
Epoch [2/5], Step [400/938], Loss: 0.0359
Epoch [2/5], Step [500/938], Loss: 0.0027
Epoch [2/5], Step [600/938], Loss: 0.0217
Epoch [2/5], Step [700/938], Loss: 0.0462
Epoch [2/5], Step [800/938], Loss: 0.2010
Epoch [2/5], Step [900/938], Loss: 0.0341
Test Accuracy after Epoch 2: 98.75%
Epoch [3/5], Step [100/938], Loss: 0.0509
Epoch [3/5], Step [200/938], Loss: 0.0254
Epoch [3/5], Step [300/938], Loss: 0.0209
Epoch [3/5], Step [400/938], Loss: 0.0117
Epoc

RuntimeError: example_kwarg_inputs should be a dict

In [ ]:
# 关闭 TensorBoard
writer.close()